In [ ]:
import numpy as np
from functools import partial

class Wavefunction():
    def __init__(self):
        self.l = (0.5 + 5063/10000) * 1e-10
        self.C = (-2 * m_e)/(h_bar**2)
    
    def euler(self, w, psi, E, V, dx):
        psi_e = psi + w*dx
        w_e = w + self.C*dx*(E - V)*psi

        return w_e, psi_e
    
    def euler_cromer(self, w, psi, E, V, dx):
        psi_ec = psi + w*dx
        w_ec = w + self.C*dx*(E - V)*psi
        return w_ec, psi_ec
    
    def runge_kutta(self, w, psi, E, V, dx):
        L = E - V # L is lagrangian
        
        apsi = w
        aw = self.C * L * psi 

        bpsi = w + (dx/2)*aw        
        bw = self.C* L *(psi + (dx/2)*apsi)

        cpsi = w + (dx/2)*bw
        cw = self.C* L *(psi + (dx/2)*bpsi)

        dpsi = w + dx*cw
        dw = self.C * L * (psi + dx*cpsi)

        w_rk = w + (1/6)*(aw + 2*bw + 2*cw + dw)*dx
        psi_rk = psi + (1/6)*(apsi + 2*bpsi + 2*cpsi + dpsi)*dx
        
        return w_rk, psi_rk
    
    def ipw_wavefunction(self, x, n):
        return math.sqrt(2/(self.l))*math.sin(n * math.pi * x/(self.l))
    
    def eigen_energy_solver(self, n, V): # method : Literal[str] = ["euler", "euler_cromer", "runge_kutta"]
        # code should read like psedocode
        
        dx = 1e-13
        
        steps = np.arange(0, self.l, dx)
        
        delta = 1
        
        e_max = 1e-16
        e_min = 1e-25
        
        converged = False

        while not converged:
            E = (e_max + e_min) * 0.5

            w = np.zeros(len(steps))
            psi = np.zeros(len(steps))
            
            # inital conditions
            
            w[0] = 1 
            psi[0] = 0
    
            for i, step in enumerate(steps[:-1]):
                
                w[i + 1], psi[i + 1] =  self.runge_kutta(w[i], psi[i], E, V(x = step), dx)
                
            roots = 0

            for i, _ in enumerate(psi[:-1]):
                if psi[i + 1] * psi[i] < 0: # number of crosses of x axis
                    roots += 1

            if roots >= n: # too high
                e_max = E
            else: # too low
                e_min = E

            delta = e_max - e_min

            if delta < 1e-30: # add proper condition.
                converged = True

            print(e_max, e_min)
            

In [2]:
# potential functions
def ipw(x):
    return 0

def harmonic(k, x):
    return 0.5 * k * x**2

def triangular(m, x):
    # region based as well, need to get formula
    return m * x

def square(bound_1, Vb1, bound_2, Vb2, x, Vs):
    if x <= bound_1:
        return Vb1

    elif x >= bound_2:
        return Vb2

    else:
        return Vs

def morse(x, r_e, a, D_e):
    '''
    Parameters
    ----------

    x: distance between particles
    r_e: equilibrium bond distance
    a: width of the well
    D_e: dissocaition energy / well depth

    Returns
    --------

    morse_potential: float
        morse potential at x given the parameters of the potential.

    '''
    return D_e * ( 1 - math.exp(-a * (x - r_e) * 1e20) )**2